In [ ]:
# Evaluación Cuantitativa de Explicabilidad con Quantus

Este notebook evalúa cuantitativamente la calidad de las explicaciones XAI usando la librería Quantus.

**Métricas evaluadas:**
- **Fidelidad (Faithfulness)**: Faithfulness Correlation - mide si la explicación refleja el comportamiento interno del modelo
- **Robustez (Robustness)**: Average Sensitivity - evalúa la estabilidad ante perturbaciones leves
- **Complejidad (Complexity)**: Entropy/Complexity - estima la simplicidad de la explicación
- **Aleatorización (Randomization)**: Randomization Test - mide la dependencia respecto a la semilla
- **Localización (Localization)**: Region Perturbation - determina la precisión espacial

**Métodos XAI evaluados:**
- Grad-CAM
- Grad-CAM++
- Integrated Gradients
- Saliency Maps

**Nota:** Este notebook evalúa los 3 modelos individuales (blood, retina, breast) y genera un gráfico radar comparativo.

(True, 'Quadro RTX 6000')

In [ ]:
# Configuración inicial

import os
import json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from math import pi

# Cambiar al directorio del proyecto
PROJECT_DIR = Path("/home/TFM_Laura_Monne")
os.chdir(PROJECT_DIR)
print(f"Directorio de trabajo: {os.getcwd()}")

# Configuración
OUTPUTS_DIR = PROJECT_DIR / "outputs"
OUTPUTS_DIR.mkdir(exist_ok=True)

# Número de muestras a evaluar (puedes ajustarlo según tus necesidades)
NUM_SAMPLES = 30  # Aumenta para mayor precisión, pero tardará más

print(f"Directorio de outputs: {OUTPUTS_DIR}")
print(f"Muestras a evaluar por dataset: {NUM_SAMPLES}")


'/home/TFM_Laura_Monne'

In [ ]:
# Ejecutar evaluación Quantus para cada dataset

datasets = ["blood", "retina", "breast"]

for dataset in datasets:
    print(f"\n{'='*60}")
    print(f"Evaluando dataset: {dataset}")
    print(f"{'='*60}")
    
    !python quantus_evaluation.py \
        --dataset {dataset} \
        --data_dir ./data \
        --num_samples {NUM_SAMPLES} \
        --methods gradcam gradcampp integrated_gradients saliency \
        --output outputs/quantus_metrics_{dataset}.json
    
    print(f"✅ Evaluación completada para {dataset}\n")

  EVALUACIÓN QUANTUS - RESNET18 XAI
Dispositivo: cuda
Métodos: ['gradcam', 'gradcampp', 'integrated_gradients', 'saliency']
Muestras a evaluar: 5
Cargando modelo desde results/best_model.pth...
Modelo ResNet-18 creado:
  - Parámetros totales: 22,362,142
  - Parámetros entrenables: 22,362,142
  - Número de clases: 15
✅ Modelo cargado correctamente.
Cargando datasets de MedMNIST...

Cargando BLOODMNIST...
  - Entrenamiento: 11959
  - Validación:    1712
  - Test:          3421
  - Clases:        8
  - Canales (orig):3 -> input: 3

Cargando RETINAMNIST...
  - Entrenamiento: 1080
  - Validación:    120
  - Test:          400
  - Clases:        5
  - Canales (orig):3 -> input: 3

Cargando BREASTMNIST...
  - Entrenamiento: 546
  - Validación:    78
  - Test:          156
  - Clases:        2
  - Canales (orig):1 -> input: 3
DataLoaders creados:
  - Train: 13585 muestras, 13585 batches
  - Val:   1910 muestras, 1910 batches
  - Test:  3977 muestras, 3977 batches
Recolectando muestras:   0%|  

In [ ]:
# Cargar resultados de los 3 datasets

results_by_dataset = {}

for dataset in datasets:
    json_path = OUTPUTS_DIR / f"quantus_metrics_{dataset}.json"
    if json_path.exists():
        with open(json_path, "r") as f:
            results_by_dataset[dataset] = json.load(f)
        print(f"✅ Cargado: {dataset} ({json_path.name})")
    else:
        print(f"⚠️  No encontrado: {json_path.name}")
        print(f"   Ejecuta primero la celda anterior para generar los resultados.")

if not results_by_dataset:
    raise ValueError("❌ No se encontraron resultados. Ejecuta la celda anterior primero.")


In [ ]:
# Extraer y normalizar métricas para el gráfico radar

def normalize_metric(value, metric_name, method_name):
    """
    Normaliza las métricas al rango [0, 1] para el gráfico radar.
    
    Para cada métrica:
    - Faithfulness: valores más altos son mejores (correlación)
    - Robustness: valores más bajos son mejores (menos sensibilidad)
    - Complexity: valores más bajos son mejores (menos complejidad)
    - Randomization: valores más bajos son mejores (menos dependencia)
    - Localization: valores más altos son mejores (mejor localización)
    """
    # Valores típicos observados (puedes ajustarlos según tus resultados)
    ranges = {
        "faithfulness": (-1.0, 1.0),  # Correlación típicamente entre -1 y 1
        "robustness": (0.0, 1.0),     # Sensibilidad típicamente entre 0 y 1
        "complexity": (0.0, 20.0),    # Complejidad típicamente entre 0 y 20
        "randomization": (0.0, 1.0),  # Randomization típicamente entre 0 y 1
        "localization": (0.0, 1.0),   # Localization típicamente entre 0 y 1
    }
    
    if metric_name not in ranges:
        return 0.5  # Valor por defecto
    
    min_val, max_val = ranges[metric_name]
    
    # Normalizar al rango [0, 1]
    normalized = (value - min_val) / (max_val - min_val) if max_val != min_val else 0.5
    normalized = max(0.0, min(1.0, normalized))  # Clamp a [0, 1]
    
    # Invertir si valores más bajos son mejores
    if metric_name in ["robustness", "complexity", "randomization"]:
        normalized = 1.0 - normalized
    
    return normalized

# Métricas a mostrar en el radar
metric_names = ["faithfulness", "robustness", "complexity", "localization"]
metric_labels = ["Fidelidad", "Robustez", "Complejidad", "Localización"]

# Métodos XAI
methods = ["gradcam", "gradcampp", "integrated_gradients", "saliency"]
method_labels = {
    "gradcam": "Grad-CAM",
    "gradcampp": "Grad-CAM++",
    "integrated_gradients": "Integrated Gradients",
    "saliency": "Saliency"
}

# Recopilar métricas normalizadas (promedio entre los 3 datasets)
normalized_metrics = {method: {metric: [] for metric in metric_names} for method in methods}

for dataset, results in results_by_dataset.items():
    for method in methods:
        if method not in results:
            continue
        for metric in metric_names:
            if metric in results[method] and results[method][metric] is not None:
                mean_val = results[method][metric].get("mean", 0.0)
                norm_val = normalize_metric(mean_val, metric, method)
                normalized_metrics[method][metric].append(norm_val)

# Promediar entre datasets
for method in methods:
    for metric in metric_names:
        values = normalized_metrics[method][metric]
        if values:
            normalized_metrics[method][metric] = np.mean(values)
        else:
            normalized_metrics[method][metric] = 0.0

print("Métricas normalizadas (promedio entre datasets):")
for method in methods:
    print(f"\n{method_labels[method]}:")
    for metric in metric_names:
        print(f"  {metric}: {normalized_metrics[method][metric]:.4f}")


In [ ]:
# Generar gráfico radar (spider chart)

# Configuración del gráfico
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

# Ángulos para cada métrica
angles = [n / float(len(metric_names)) * 2 * pi for n in range(len(metric_names))]
angles += angles[:1]  # Cerrar el círculo

# Colores para cada método
colors = {
    "gradcam": "#FF6B6B",  # Rojo
    "gradcampp": "#9B59B6",  # Morado
    "integrated_gradients": "#8B4513",  # Marrón
    "saliency": "#808080"  # Gris
}

# Dibujar cada método
for method in methods:
    values = [normalized_metrics[method][metric] for metric in metric_names]
    values += values[:1]  # Cerrar el círculo
    
    ax.plot(angles, values, 'o-', linewidth=2, label=method_labels[method], color=colors[method])
    ax.fill(angles, values, alpha=0.25, color=colors[method])

# Configurar etiquetas
ax.set_xticks(angles[:-1])
ax.set_xticklabels(metric_labels, fontsize=12)

# Configurar el rango radial
ax.set_ylim(0, 1)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=10)
ax.grid(True, linestyle='--', alpha=0.5)

# Título
plt.title('Perfil de Métricas por Método XAI (Normalizado 0-1)', 
          size=16, fontweight='bold', pad=20)

# Leyenda
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=11)

# Guardar figura
output_path = OUTPUTS_DIR / "quantus_radar_chart.png"
plt.tight_layout()
plt.savefig(output_path, dpi=300, bbox_inches='tight')
print(f"✅ Gráfico guardado en: {output_path}")

plt.show()


In [ ]:
# Tabla resumen de métricas (valores originales, no normalizados)

summary_rows = []

for dataset, results in results_by_dataset.items():
    for method in methods:
        if method not in results:
            continue
        for metric in metric_names:
            if metric in results[method] and results[method][metric] is not None:
                mean_val = results[method][metric].get("mean", 0.0)
                std_val = results[method][metric].get("std", 0.0)
                summary_rows.append({
                    "Dataset": dataset.upper(),
                    "Método": method_labels[method],
                    "Métrica": metric_labels[metric_names.index(metric)],
                    "Media": f"{mean_val:.4f}",
                    "Std": f"{std_val:.4f}",
                })

df_summary = pd.DataFrame(summary_rows)
print("📊 Resumen de métricas por dataset y método:")
display(df_summary)

# Guardar tabla
csv_path = OUTPUTS_DIR / "quantus_metrics_summary.csv"
df_summary.to_csv(csv_path, index=False)
print(f"\n✅ Tabla guardada en: {csv_path}")
